In [5]:
import os
import pandas as pd
from utils.utils import get_timetsamp_with_random
from utils.map_utils import get_game_info_with_G_eval
from utils.eval_gpt_utils import get_csv,eval_game
import os
import os.path as osp
import networkx as nx
import json
import matplotlib.pyplot as plt
from utils.map_utils import get_game_info_with_G_eval
from utils.map_utils_old import find_all_paths
from utils.clean_utils import compute_hash_for_path
import tiktoken
import openai
import time
import traceback
import pandas as pd

In [6]:
result_dir='/share/data/mei-work/kangrui/github/mango/kangrui/data/gpt-games-results-clean-new-new'
map_dir='/share/data/mei-work/kangrui/github/mango/data/'

In [7]:
def get_exceeded_cutoff(walkthrough:str):
    cut_off_number = int(walkthrough.split('NUM: ')[-2].split('\n')[0])
    if cut_off_number>70:
        return cut_off_number-70
    else:
        return 0

def get_cut_off_and_walkthrough_text(walkthrough:str,token_size_limit=3600,model_name='gpt-4',cut_off_number=None):
    encoder = tiktoken.encoding_for_model(model_name)
    enc = encoder.encode(walkthrough)
    if len(enc) > token_size_limit:
        cut_off_walkthrough_text = encoder.decode(enc[:token_size_limit])
    else:
        cut_off_walkthrough_text = encoder.decode(enc)
        
    if cut_off_number is None:
        
        cut_off_number = int(cut_off_walkthrough_text.split('NUM: ')[-2].split('\n')[0])
        if cut_off_number > 70:
            cut_off_number = 70
    

    walkthrough_text = walkthrough.split('NUM: {}'.format(cut_off_number + 1))[0]

    return cut_off_number,len(encoder.encode(walkthrough_text))

def normalized_edit_distance(s1, s2):
    s1 = s1.lower()
    s2 = s2.lower()
    
    s1=s1.split()
    s2=s2.split()
    
    m = len(s1) + 1
    n = len(s2) + 1

    dp = [[0] * n for _ in range(m)]

    for i in range(m):
        dp[i][0] = i

    for j in range(n):
        dp[0][j] = j

    for i in range(1, m):
        for j in range(1, n):
            if s1[i - 1] == s2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = min(
                    dp[i - 1][j] + 1,  # deletion
                    dp[i][j - 1] + 1,  # insertion
                    dp[i - 1][j - 1] + 1  # substitution
                )
    
    # Compute the normalized score
    max_len = max(len(s1), len(s2))
    score = 1 - dp[m - 1][n - 1] / max_len
    return score

def get_edegs(G):
    edges=[]
    for edge in G.edges(data=True):
        edges.append(edge)
    return edges
def num_of_locations(G,cut_off_number):
    edges=get_edegs(G)
    locations=set()
    for edge in edges:
        if edge[2]['step_min_cutoff']<=cut_off_number:
            locations.add(edge[0])
            locations.add(edge[1])
    return len(locations)

def num_of_edges(G,cut_off_number):
    edges=get_edegs(G)
    cnt=0
    for edge in edges:
        if edge[2]['step_min_cutoff']<=cut_off_number:
            cnt+=1         
    return cnt

def num_of_exp_edges(G,cut_off_number):
    edges=get_edegs(G)
    cnt=0
    for edge in edges:
        if edge[2]['step_min_cutoff']<=cut_off_number and edge[2]['seen_in_forward']:
            cnt+=1         
    return cnt
def num_of_exp_edges(G,cut_off_number):
    edges=get_edegs(G)
    cnt=0
    for edge in edges:
        if edge[2]['step_min_cutoff']<=cut_off_number and edge[2]['seen_in_forward']:
            cnt+=1         
    return cnt

def num_of_imp_edges(G,cut_off_number):
    edges=get_edegs(G)
    cnt=0
    for edge in edges:
        if edge[2]['step_min_cutoff']<=cut_off_number and (not edge[2]['seen_in_forward']):
            cnt+=1         
    return cnt

def ratio_of_conf_locations(G,all2all,cut_off_number):
    edges=get_edegs(G)
    locations=set()
    for edge in edges:
        if edge[2]['step_min_cutoff']<=cut_off_number:
            locations.add(edge[0])
            locations.add(edge[1])
    locations=list(locations)
    
    score=0
    for i in range(len(locations)):
        current_max_score = float('-inf')
        for j in range(len(locations)):
            if i != j:  # Don't compare an item with itself
                current_score = normalized_edit_distance(locations[i], locations[j])
                if current_score > current_max_score:
                    current_max_score = current_score
        score+=current_max_score
        
    return score/len(locations) if len(locations)>0 else 0

def num_of_conf_locations(G,all2all,cut_off_number):
    edges=get_edegs(G)
    locations=set()
    for edge in edges:
        if edge[2]['step_min_cutoff']<=cut_off_number:
            locations.add(edge[0])
            locations.add(edge[1])
    locations=list(locations)
    
    score=0
    for i in range(len(locations)):
        current_max_score = float('-inf')
        for j in range(len(locations)):
            if i != j:  # Don't compare an item with itself
                current_score = normalized_edit_distance(locations[i], locations[j])
                if current_score > current_max_score:
                    current_max_score = current_score
        score+=current_max_score
        
    return score
    
def num_of_df(G,all2all,cut_off_number):
    idset=set()
    cnt=0
    for path in all2all:
        if path['path_min_cutoff']<=cut_off_number:
            idset.add(path['id'])
            cnt+=1
    assert len(idset)==cnt
    return cnt
            
def num_of_rf(G,all_pairs,cut_off_number):
    idset=set()
    cnt=0
    for pair in all_pairs:
        if min(pair['path_min_cutoffs'])<=cut_off_number:
            idset.add(pair['id'])
            cnt+=1
    assert len(idset)==cnt
    return cnt

def average_length_of_all2all(all2all,cut_off_number):
    length=0
    cnt=0
    for path in all2all:
        if path['path_min_cutoff']<=cut_off_number:
            length+=path['step_count']
            cnt+=1
    return length/cnt if cnt>0 else 0

def average_length_of_all2all_simple(all2all,cut_off_number):
    length=0
    cnt=0
    for path in all2all:
        if path['path_min_cutoff']<=cut_off_number and path['all_steps_seen_in_forward']:
            length+=path['step_count']
            cnt+=1
    return length/cnt if cnt>0 else 0

def average_length_of_all2all_hard(all2all,cut_off_number):
    length=0
    cnt=0
    for path in all2all:
        if path['path_min_cutoff']<=cut_off_number and not path['all_steps_seen_in_forward']:
            length+=path['step_count']
            cnt+=1
    return length/cnt if cnt>0 else 0
        
def average_num_of_imp_edge(all2all,cut_off_number):
    num=0
    cnt=0
    for path in all2all:
        if path['path_min_cutoff']<=cut_off_number and not path['all_steps_seen_in_forward']:
            cnt+=1
            for edge in path["path_details"]:
                if not edge["seen_in_forward"]:
                    num+=1
                    
    return num/cnt if cnt>0 else 0

def num_of_special_moves(G,cut_off_number):
    
    reverse_dict = {
    "up": "down",
    "down": "up",
    "north": "south",
    "south": "north",
    "east": "west",
    "west": "east",
    "northeast": "southwest",
    "northwest": "southeast",
    "southeast": "northwest",
    "southwest": "northeast"
}
    
    edges=get_edegs(G)
    actions=set()
    for edge in edges:
        if edge[2]['step_min_cutoff']<=cut_off_number:
            if edge[2]['action'] not in reverse_dict.keys():
                actions.add(edge[2]['action'])
    return len(actions)

def num_of_tokens_per_edge(G,cut_off_number,token_num):
    return token_num/num_of_exp_edges(G,cut_off_number)

def analyze_map(map_dir,game_name,cut_off_number=None):
    G_eval,G,actions,locations,all2all,all_pairs,walkthrough=get_game_info_with_G_eval(map_dir,game_name)
    cut_off_number,token_num=get_cut_off_and_walkthrough_text(walkthrough,cut_off_number=cut_off_number)
    
    result = {
        'exceeded_cutoff':get_exceeded_cutoff(walkthrough)
    }
    return result

In [8]:
map_dict1={}
for game_name in os.listdir(map_dir):
    map_dict1[game_name]=analyze_map(map_dir,game_name,cut_off_number=10000)
string_data1 = json.dumps(map_dict1)

In [9]:
map_dict1

{'inhumane': {'exceeded_cutoff': 51},
 'zenon': {'exceeded_cutoff': 12},
 'yomomma': {'exceeded_cutoff': 27},
 'ztuu': {'exceeded_cutoff': 13},
 'adventureland': {'exceeded_cutoff': 99},
 'balances': {'exceeded_cutoff': 51},
 'wishbringer': {'exceeded_cutoff': 113},
 'enchanter': {'exceeded_cutoff': 194},
 'afflicted': {'exceeded_cutoff': 27},
 'awaken': {'exceeded_cutoff': 0},
 'hhgg': {'exceeded_cutoff': 290},
 'lostpig': {'exceeded_cutoff': 75},
 'hollywood': {'exceeded_cutoff': 326},
 'advent': {'exceeded_cutoff': 206},
 '905': {'exceeded_cutoff': 0},
 'partyfoul': {'exceeded_cutoff': 0},
 'snacktime': {'exceeded_cutoff': 0},
 'spirit': {'exceeded_cutoff': 1193},
 'pentari': {'exceeded_cutoff': 0},
 'huntdark': {'exceeded_cutoff': 0},
 'loose': {'exceeded_cutoff': 0},
 'seastalker': {'exceeded_cutoff': 133},
 'library': {'exceeded_cutoff': 0},
 'omniquest': {'exceeded_cutoff': 7},
 'jewel': {'exceeded_cutoff': 152},
 'cutthroat': {'exceeded_cutoff': 265},
 'infidel': {'exceeded_cut

In [12]:
cnt=0
total=0
for k,v in map_dict1.items():
    cnt+=1
    total+=v['exceeded_cutoff']
print(total/cnt)

183.0566037735849
